## Limpeza de dados

Uma parte muito importante do data wrangling, etapa que limpamos e corrigimos dados no processo de análise, é a limpeza dos dados que não queremos ou estão incompletos

Neste episódio vou mostrar desde a identificação de dados incorretos até onde decidimos o que fazer com os mesmos

Vamos utilizar basicamente o pandas como Jupyter Notebook

Lembrando que fiz um post exclusivamente para identificar dados nulos, veja aqui: https://medium.com/matheusbudkewicz/data-science-tips-01-identificar-dados-faltantes-no-dataset-8ca67fe19c4c

Outra observação importante é que estamos utilizando a linguagem Python na lib pandas, mas esta etapa pode ser feita em outras partes do processo, por exemplo em SQL quando obtemos os dados de uma base

## Datasets

Vamos utilizar dois conjuntos de dados, para duas aplicações distintas de limpeza

O primeiro tem dados de chamadas de emergência, veja aqui:
    
https://www.kaggle.com/mchirico/montcoalert/version/49#911.csv

O segundo é o famosão do Titanic:

https://github.com/awesomedata/awesome-public-datasets/tree/master/Datasets


## Iniciando

Vamos importar as libs que usaremos, depois o dataset e testar se está funcionando

In [1]:
# Importando as libs
import pandas as pd
import numpy as np
import math

# Lendo o dataset
df = pd.read_csv('911.csv')

# Testando se está tudo certo
df.head()

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1


## Identificando linhas nulas

Como já abordada no artigo anterior, onde explico mais a fundo como funciona, vamos utilizar o metodo isnull junto de sum para identificar os dados faltantes

In [2]:
# Identificando dados faltantes
df.isnull().sum()

lat              0
lng              0
desc             0
zip          39590
title            0
timeStamp        0
twp            115
addr             0
e                0
dtype: int64

## O que sabemos até agora:
    
Bom até aqui vemos que mais de 39 mil colunas de zip code estão nulas, e também que 115 de twp, que é referente as cidades, também estão vazias

Vamos criar um caso que a coluna de cidades é essencial para nossa análise, então linhas que tem este dado faltante devem ser excluídas, então vamos lá limpar as linhas sem cidade

In [3]:
# Limpando dados nulos de uma coluna específica
df = df.dropna(subset=['twp'])

## Explicando o método

Utilizamos o metodo dropna para fazer as alterações, agora vamos entender como funciona

df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

- axis: Se for 0 'dropa' as linhas que contem os dados faltantes, caso 1 remove a coluna inteira que tem dados faltantes;
- how: Determina se a coluna vai ser removida do DataFrame, se passarmos 'any' como valor do parâmetro se tiver apenas um valor faltante que seja a coluna é removida, se passarmos 'all' precisamos que todos os valores sejam faltantes/nulos para que a coluna seja removida;
- thresh: com um inteiro, podemos definir o mínimo de colunas de NaN aceitáveis para começar a dropar, ex: caso tenha thresh = 50, precisamos que a coluna tenha mais 50 de valorers faltantes para o método ser aplicado;
- subset: se estamos removendo linhas, podemos inserir um array de colunas onde o metodo será aplicado;
- inplace: caso seja True aplica as alterações no dataset de forma automática;



## O que aconteceu?

Bom, executamos o metodo dropna no dataset, com o parametro subset para escolher a coluna, assim não mexemos nos dados de zip code

Vamos observar as mudanças

In [4]:
# Identificando dados faltantes, após limpar twp
df.isnull().sum()

lat              0
lng              0
desc             0
zip          39524
title            0
timeStamp        0
twp              0
addr             0
e                0
dtype: int64

## Eliminando todos os dados NaN do dataset:

Se na verdade todos os dados NaN podem atrapalhar nossa análise, simplesmente podemos chamar o metodo no dataset

Alem disso podemos utilizar o parâmetro inplace = True para não termos que igualar a variável, assim as mudanças já serão salvas automaticamente, veja:

In [5]:
# Limpando todos os dados nulos do dataset
df.dropna(inplace=True)

In [6]:
# Identificando dados faltantes, após limpar tudo e testando o parâmetro inplace
df.isnull().sum()

lat          0
lng          0
desc         0
zip          0
title        0
timeStamp    0
twp          0
addr         0
e            0
dtype: int64

## Conclusão:

Vimos que podemos tanto eliminar linhas com dados faltantes por coluna ou o dataset inteiro

Há também outros parâmetros que podem nos ajudar, como o inplace

## Outro caso

Vamos reiniciar nossa variável df com outro dataset, para aplicarmos agora uma tecnica de substituição/edição e não remoção como vimos anteriormente

In [7]:
# Importando nosso próximo dataset
df = pd.read_csv('titanic.csv')

# Testando a importação
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# Identificando dados faltantes
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Iniciando pela idade

Como podemos ver há 177 linhas com idades nulas

Em nosso caso, não podemos deletar nenhuma linha, pois as informações são todas fundamentais

O que poderiamos fazer neste caso?

Bom, optaremos por calcular a média da coluna idade e substituir os dados nulos/NaN por este valor calculado

In [9]:
# Descobrindo a idade média
idadeMedia = df['Age'].mean()

# Arredondando a idade média
idadeMedia = math.floor(idadeMedia)

print('A idade média deste dataset é {} anos'.format(idadeMedia))

A idade média deste dataset é 29 anos


## Ajustando o valor

Como este valor quebrado não nos interessa muito, pois precisamos de um número inteiro, arredondamos o valor com auxílio da biblioteca math e o método floor

E depois disso substituir os valores nulos

In [10]:
# Dando update nos valores nulos em Age
df.update(df['Age'].fillna(idadeMedia))

# Identificando dados faltantes novamente
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Substituição concluída

Como podemos verificar, as linhas de age foram substituidas com sucesso

Com esta verificação abaixo já possível identificar algumas células com a idade média de 29

In [11]:
# Usando loc para verificar idades medias no dataset
df.loc[df['Age'] == 29].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.0,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,29.0,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,29.0,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,29.0,0,0,330959,7.8792,NaN,Q


In [12]:
# Contando ocorrências de idade média
df['Age'].loc[df['Age'] == 29].count()

197

## Próximo caso

Este é semelhante ao primeiro, vamos supor que precisamos de algum dado na coluna cabine diferente de NaN

Pois na hora de apresentar os dados é importante para nosso público alvo que eles saibam a inexistência do cadastro das cabines

Então vamos substituir elas com um valor fixo, por exemplo 'Não cadastrada'

In [13]:
# Dando update nos valores da coluna Cabin
df.update(df['Cabin'].fillna('Não cadastrada'))

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Não cadastrada,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Não cadastrada,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Não cadastrada,S


## Conclusão

Creio que nesses 4 casos que abordei, já podemos ter uma ideia do que fazer para tratar os dados nulos

São algumas das ações que podemos tomar:
    
- Dropar a coluna que contenha dados nulos;
- Dropar apenas as linhas com dados nulos;
- Substituir pela média da coluna;
- Substituir por um valor fixo;

Destrinchamos o método dropna, abordando todos os seus parâmetros e também vimos a importância geral do inplace, que pode alterar o dataset no ato da execução do método

Lembrando que essa parte do processo de data wrangling é decisiva, é o que vai tornar nossa análise boa ou ruim, pois estes dados mal cadastrados podem comprometer todo o processo

Obrigado a todos que leram, e qualquer feedback será bem-vindo :)